# Dependency Parsing




In [43]:
# Global dependencies here
from typing import Dict, List, Set, Iterable, Sequence, Tuple
from collections import deque
import itertools
from copy import deepcopy
import json

%env JOBLIB_TEMP_FOLDER=/tmp

## Arc-eager transition-based parsing

In [83]:
class TransitionBasedParser(object):
    
    class Arc(object):
        
        def __init__(self, left_word: int, label: str, right_word: int,
                     sentence: List[str]):
            # sentence is required to start with "ROOT"
            self.left = left_word
            self.label = label
            self.right = right_word
            self.sentence = sentence
            
        def __repr__(self):
            left = self.sentence[self.left]
            right = self.sentence[self.right]
            return f"({left} --|{self.label}|--> {right})"

        
    class Configuration(object):
        
        def __init__(self, stack: List[int], buffer: List[int],
                     arcs: List, sentence: List[str], features=None):
            self.stack = stack
            self.buffer = deque(buffer)
            self.arcs = arcs
            self.sentence = sentence
            self.features = features or []

        def __repr__(self) -> str:
            stack = [self.sentence[i] for i in self.stack]
            buffer = [self.sentence[i] for i in self.buffer]

            return f"<CONFIGURATION>\n  stack={stack}\n  buffer={buffer}\n  arcs={self.arcs}\n</CONFIGURATION>"
        
        
    class Graph(object):
        
        def __init__(self, arcs: List=None, features=None):
            self.arcs = arcs or []
            self.features = features or []

        def starts_with(self, index):
            return filter(lambda arc: arc.left == index, self.arcs)
                
        def ends_with(self, index):
            return filter(lambda arc: arc.right == index, self.arcs)
        
        def contains_arc(self, left: int, right: int):
             return filter(lambda arc:
                               arc.left == left
                               and arc.right == right, 
                           self.arcs)
            
        def contains_arc_label(self, left: int, label: str, right: int):
             return filter(lambda arc:
                               arc.left == left
                               and arc.right == right
                               and arc.label == label, 
                           self.arcs)
        
        def __repr__(self) -> str:
            return str(self.arcs)
        
        def subset(self, other):
            # Are all the arcs in the other graph in this one?
            a = set([(a.left, a.label, a.right) for a in self.arcs])
            b = set([(a.left, a.label, a.right) for a in other])

            return b.difference(a)
                    

    
    def __init__(self, labels: List[str], ml_data=None):
        self.labels = labels
        self.ml_data = ml_data
    
    def parse(self, sentence: List[str], features):
        # Accepts a list of tokens
        configuration = self._initialization(sentence)
        configuration.features = features
        print("Initial configuration:")
        print(configuration)

        # Keep making transitions until the termination
        # requirement is met.
        while not self._check_termination(configuration):
            print()
            configuration = self._make_transition(configuration)
            print("New configuration:")
            print(configuration)
        
        return configuration.arcs
    
    def _make_transition(self, c):
        # Get features for the configuration
        #     "vec": vectorizer, "feat": featurizer, "clf": classifier

        features = self.ml_data['feat'].featurize(c)
        
        # Now turn them into a vector
        vector = self.ml_data['vec'].transform(features)
        
        prediction = self.ml_data['clf'].predict(vector)[0]
        print(prediction)
        if prediction == "LA":
            print("Left arc")
            new_configuration = self._left_arc(self.labels[0], c)
        elif prediction == "RA":
            print("Right arc")
            new_configuration = self._right_arc(self.labels[0], c)
        elif prediction == "SH":
            print("Shift")
            new_configuration = self._shift(c)
        else:
            print("Reduce")
            new_configuration = self._reduce(c)
            
        if not new_configuration:
            print("Fallback Shift")
            new_configuration = self._shift(c)
        
        return new_configuration
    
    def _make_transition_naive(self, c):
        # Two big questions:
        # How to chose the correct label?
        # How to chose the correct transition
        label = self.labels[0]
        new_configuration = None
             
        if not new_configuration:
            new_configuration = self._reduce(c)
            
        if not new_configuration:
            new_configuration = self._left_arc(label, c)

        if not new_configuration:
            new_configuration = self._shift(c)
            
        if not new_configuration:
            new_configuration = self._right_arc(label, c)  
        
        return new_configuration
    
    def _left_arc(self, label: str, c):
        c = deepcopy(c)
                
        if len(c.stack) == 0:
            return None
        
        # Remove from stack
        w_i = c.stack.pop()
        
        # Keep on buffer
        w_j = c.buffer[0]
        
        if w_i is None or w_j is None:
            return None
        
        # Create arc: w_j -> w_i
        arc = TransitionBasedParser.Arc(left_word=w_j, label=label, right_word=w_i, sentence=c.sentence)
        c.arcs.append(arc)

        return c
    
    def _right_arc(self, label: str, c):
        c = deepcopy(c)
        
        if len(c.stack) == 0:
            return None
        
        # Keep on stack
        w_i = c.stack[-1]
        
        # Remove from buffer
        w_j = c.buffer.popleft()
        
        if w_i is None or w_j is None:
            return None
        
        # Create arc: w_i -> w_j
        arc = TransitionBasedParser.Arc(left_word=w_i, label=label, right_word=w_j, sentence=c.sentence)
        c.arcs.append(arc)
        
        # Append wj to stack
        c.stack.append(w_j)

        return c
    
    def _reduce(self, c):
        c = deepcopy(c)
        
        if len(c.stack) == 0:
            return None
        
        w_i = c.stack[-1]
        
        # Seach for an arc leading to w_i
        valid_arc = list(filter(lambda arc: arc.right == w_i, c.arcs))
        
        if not valid_arc:
            return None
        
        # Remove w_i from the stack
        c.stack.pop()
        
        return c
    
    def _shift(self, c):        
        
        c = deepcopy(c)
        c.stack.append(c.buffer.popleft())
        return c
    
    def _initialization(self, sentence: List[str]):
        return TransitionBasedParser.Configuration(
            stack=[0], buffer=list(range(1, len(sentence))),
            arcs=[], sentence=sentence)
    
    def _check_termination(self, c) -> bool:
        return not c or not c.buffer
    
    

In [3]:
import requests

# Donload training dataset
f = requests.get("https://www.dropbox.com/s/nnxvjk1n3wc5sof/ewt-train.conll?dl=1")

corpus = f.text

## Read Corpus in the CONLL format

In [12]:
from conllu.parser import parse

class ConllTreeBankReader(object):
        
    @staticmethod
    def parse(content: str):
        # Iterates over text in CONLL format and generates
        # Graphs with the arcs of the dependency tree
        Arc = TransitionBasedParser.Arc
        Graph = TransitionBasedParser.Graph

        for sent in parse(content):
            """
            conllu output:
            
            OrderedDict([
                ('id', 1),
                ('form', 'The'),
                ('lemma', 'the'),
                ('upostag', 'DET'),
                ('xpostag', 'DT'),
                ('feats', OrderedDict([('Definite', 'Def'), ('PronType', 'Art')])),
                ('head', 4),
                ('deprel', 'det'),
                ('deps', None),
                ('misc', None)
            ])
            """
            
            sentence_str = ["ROOT"] + [t['form'] for t in sent]
            
            yield Graph([Arc(
                left_word=t['head'],
                label=t['deprel'],
                right_word=t['id'],
                sentence=sentence_str
            ) for t in sent],
            features=sent)
            

## Turn a dependency graph into a sequence of configurations

In [22]:
class ConfigurationExtractor(object):
    
    @staticmethod
    def generate_configurations(graphs: Sequence[TransitionBasedParser.Graph]):
        Config = TransitionBasedParser.Configuration
        Graph = TransitionBasedParser.Graph
        parser = TransitionBasedParser(None)
        
        for graph in graphs:
            # Skip sentences without arcs
            if not graph.arcs:
                continue
                                
            sentence = graph.arcs[0].sentence
            features = graph.features
            
            """ Generate initial configuration:
            
            For every instance (S_d , G_d) ∈ D, we ﬁrst construct a transition sequence 
            C_(0,m) = (c_0, c_1, ... , c_m) such that

                1. c_0 = c_0(S_d), <====
                2. G_d = (V_d, A_c_m).
            """
            
            configuration = parser._initialization(sentence)
            configuration.features = features
            yield "INIT", None, configuration
            
            """ Use the oracle function to generate all
            non-terminal configurations:
        
            See: Algorithm 1 in:
            "A Dynamic Oracle for Arc-Eager Dependency Parsing"
            """
            
            i = 500
            while not parser._check_termination(configuration):
                # assert not graph.subset(configuration.arcs)
                
                # Make sure not to end up in an infinite loop
                i -= 1
                if i <= 0:
                    break
                    
                # For all arcs, we need the buffer and stack to be not empty
                if len(configuration.stack) > 0 and len(configuration.buffer) > 0:
                    buffer_0 = configuration.buffer[0]
                    stack_0 = configuration.stack[-1]

                    # Check for left-arc:
                    # if (β[0], r, σ[0]) ∈ A_d
                    arcs = list(graph.contains_arc(buffer_0, stack_0))
                    if arcs:
                        assert len(arcs) == 1
                        label = arcs[0].label
                        configuration = parser._left_arc(label, configuration)
                        configuration.features = features
                        yield "LA", label, configuration
                        continue

                    # Check for right-arc:
                    # if (σ[0], r, β[0]) ∈ A_d
                    arcs = list(graph.contains_arc(stack_0, buffer_0))
                    if arcs:
                        assert len(arcs) == 1
                        label = arcs[0].label
                        configuration = parser._right_arc(label, configuration)
                        configuration.features = features
                        yield "RA", label, configuration
                        continue
                    
                    # Check for reduce
                    success = False
                    for k in range(stack_0):
                        arcs = list(graph.contains_arc(k, buffer_0))
                        arcs += list(graph.contains_arc(buffer_0, k))
                        if arcs:
                            if parser._reduce(configuration) is None:
                                break
                            
                            configuration = parser._reduce(configuration)
                            configuration.features = features
                            yield "RE", None, configuration
                            success = True
                            break
                    if success:
                        continue
                            
                # Else: Shift
                configuration = parser._shift(configuration)
                configuration.features = features
                yield "SH", None, configuration
        

## Turn a configuration into a FeatureDict

In [51]:
class Featurizer(object):
    
    @staticmethod
    def get_from_list(l, pos, sent):
        if (pos >= 0 and pos < len(l)) or (pos < 0 and len(l) + pos >= 0):
            return sent[l[pos]]
        else:
            return -1
        
    @staticmethod
    def featurize(c):
        s = c.sentence
        f = c.features
        a = c.arcs
        g = TransitionBasedParser.Graph(a)
        
        lemmas = ["ROOT"] + [x['lemma'] for x in f]
        pos = ["ROOT"] + [x['upostag'] for x in f]
        deps = ["-"] + [x['deprel'] for x in f]
        
        outgoing = []
        for x in range(len(s)):
            for arc in g.starts_with(x):
                outgoing.append(arc.right)
                continue
            outgoing.append(None)
        
        incoming = []
        for x in range(len(s)):
            for arc in g.ends_with(x):
                incoming.append(arc.left)
                continue
            incoming.append(None)

        return {
#             "first_word_on_stack": Featurizer.get_from_list(c.stack, -1, s),
            "first_lemma_on_stack": Featurizer.get_from_list(c.stack, -1, lemmas),
            "first_pos_on_stack": Featurizer.get_from_list(c.stack, -1, pos),
            "first_dep_on_stack": Featurizer.get_from_list(c.stack, -1, deps),
            "first_on_stack_incoming": Featurizer.get_from_list(incoming, c.stack[-1], pos) 
                if c.stack and incoming[c.stack[-1]] is not None else "-",
            "first_on_stack_outgoing": Featurizer.get_from_list(outgoing, c.stack[-1], pos) 
                if c.stack and outgoing[c.stack[-1]]is not None else "-",
            
#             "second_word_on_stack": Featurizer.get_from_list(c.stack, -2, s),
            "second_lemma_on_stack": Featurizer.get_from_list(c.stack, -2, lemmas),
            "second_pos_on_stack": Featurizer.get_from_list(c.stack, -2, pos),
            "second_dep_on_stack": Featurizer.get_from_list(c.stack, -2, deps),

#             "first_word_on_buffer": Featurizer.get_from_list(c.buffer, 0, s),
            "first_lemma_on_buffer": Featurizer.get_from_list(c.buffer, 0, lemmas),
            "first_pos_on_buffer": Featurizer.get_from_list(c.buffer, 0, pos),
            "first_dep_on_buffer": Featurizer.get_from_list(c.buffer, 0, deps),
             "first_on_buffer_incoming": Featurizer.get_from_list(incoming, c.buffer[0], pos) 
                if c.buffer and incoming[c.buffer[0]] is not None else "-",
            "first_on_buffer_outgoing": Featurizer.get_from_list(outgoing, c.buffer[0], pos) 
                if c.buffer and outgoing[c.buffer[0]]is not None else "-",
            
#             "second_word_on_buffer": Featurizer.get_from_list(c.buffer, 1, s),
            "second_lemma_on_buffer": Featurizer.get_from_list(c.buffer, 1, lemmas),
            "second_pos_on_buffer": Featurizer.get_from_list(c.buffer, 1, pos),
            "second_dep_on_buffer": Featurizer.get_from_list(c.buffer, 1, deps),
        }
           
# for i, x in enumerate(configs):
#     if i > 1:
#         break
#     print(json.dumps(Featurizer.featurize(x), indent=2, sort_keys=True))


{
  "first_dep_on_buffer": "punct",
  "first_dep_on_stack": "root",
  "first_lemma_on_buffer": "-",
  "first_lemma_on_stack": "Al",
  "first_on_buffer_incoming": "-",
  "first_on_buffer_outgoing": "-",
  "first_on_stack_incoming": "ROOT",
  "first_on_stack_outgoing": "-",
  "first_pos_on_buffer": "PUNCT",
  "first_pos_on_stack": "PROPN",
  "second_dep_on_buffer": "flat",
  "second_dep_on_stack": "-",
  "second_lemma_on_buffer": "Zaman",
  "second_lemma_on_stack": "ROOT",
  "second_pos_on_buffer": "PROPN",
  "second_pos_on_stack": "ROOT"
}
{
  "first_dep_on_buffer": "flat",
  "first_dep_on_stack": "punct",
  "first_lemma_on_buffer": "Zaman",
  "first_lemma_on_stack": "-",
  "first_on_buffer_incoming": "PROPN",
  "first_on_buffer_outgoing": "-",
  "first_on_stack_incoming": "-",
  "first_on_stack_outgoing": "PUNCT",
  "first_pos_on_buffer": "PROPN",
  "first_pos_on_stack": "PUNCT",
  "second_dep_on_buffer": "punct",
  "second_dep_on_stack": "root",
  "second_lemma_on_buffer": ":",
  "sec

## Preparing training data

### Read the corpus and turn the configurations into feature dicts. 

This can take up to 10-15 minutes, depending on the size of the corpus.

In [50]:
configs = []
y = []

for i, (transition, label, config) in enumerate(
        ConfigurationExtractor.generate_configurations(
            ConllTreeBankReader.parse(corpus))):
    if not config or transition == "INIT":
        continue
        
    configs.append(config)
    
    if not label:
        label = ""
        
    y.append(transition)

In [53]:
feature_dicts = [Featurizer.featurize(x) for x in configs]

### Now create the actual numpy feature vectors using a dict vectorizer

In [55]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.utils import shuffle

# Todo: Shuffle data

v = DictVectorizer()
X = v.fit_transform(feature_dicts)

## Cross-valdation to find the best classifier

### Warning: This might take hours :(

In [58]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

classifiers_to_test = [
    DummyClassifier(strategy='uniform'),
    LogisticRegression(),
    LinearSVC(),
    BernoulliNB(),
    RandomForestClassifier(),
    Perceptron(n_iter=100),
#     KNeighborsClassifier(weights="distance"), # <- theres sth wrong with it Oo
]

from sklearn import cross_validation
for clf in classifiers_to_test:
    print("\n", clf)
    # Train 10 times, each time holding 10% of the data out to validate on.
    # The overall accuracy is then an average of the 10 training results.
    scores = cross_validation.cross_val_score(clf, X, y, cv=10, scoring="accuracy", n_jobs=-1)
    print("Avg. accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() / 2))


 DummyClassifier(constant=None, random_state=None, strategy='uniform')
Avg. accuracy: 0.2496 (+/- 0.0011)

 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Avg. accuracy: 0.8008 (+/- 0.0045)

 LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
Avg. accuracy: 0.7896 (+/- 0.0056)

 BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
Avg. accuracy: 0.7155 (+/- 0.0060)

 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_s

TypeError: catching classes that do not inherit from BaseException is not allowed

### Results (accuracy)

| Run | Baseline | LogisticRegression | SVC | Bernoulli Naive Bayes | Random Forest  | Perceptron | Nearest Neighbor |
|-----|----------|--------------------|-----|-----------------------|----------------|------------|------------------|
|  1  | 0.2496  | 0.8008 | 0.7896 | 0.7155 | **0.8207** |  0.7415 | | |


Run 1: Features (example): 
```
["first_dep_on_buffer", "first_dep_on_stack", "first_lemma_on_buffer", "first_lemma_on_stack", "first_on_buffer_incoming", "first_on_buffer_outgoing", "first_on_stack_incoming", "first_on_stack_outgoing",  "first_pos_on_buffer", "first_pos_on_stack", "second_dep_on_buffer", "second_dep_on_stack", "second_lemma_on_buffer",  "second_lemma_on_stack", "second_pos_on_buffer", "second_pos_on_stack"]

```

## Train the final model

In [59]:
from sklearn.linear_model import Perceptron, LogisticRegression

clf = LogisticRegression(n_jobs=-1)
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

# THE FINAL PARSER!

In [84]:
vectorizer = v
featurizer = Featurizer
classifier = clf

parser = TransitionBasedParser(["label"], ml_data={
    "vec": vectorizer, "feat": featurizer, "clf": classifier
})

In [85]:
from conllu.parser import parse

sent = """
1	He	he	PRON	PRON	_	_	_	_	_
2	worked	work	VERB	VERB	_	_	_	_	_
3	for	for	ADP	ADP	_	_	_	_	_
4	the	the	DET	DET	_	_	_	_	_
5	BBC	BBC	PROPN	PROPN	_	_	_	_	_
6	for	for	ADP	ADP	_	_	_	_	_
7	a	a	DET	DET	_	_	_	_	_
8	decade	decade	NOUN	NOUN	_	_	_	_	_
9	.	.	PUNCT	PUNCT	_	_	_	_	_
"""

features_test = parse(sent)

print(features_test)

parser.parse(["ROOT"] + [x['form'] for x in features_test[0]], features_test[0])

[[OrderedDict([('id', 1), ('form', 'He'), ('lemma', 'he'), ('upostag', 'PRON'), ('xpostag', 'PRON'), ('feats', None), ('head', None), ('deprel', '_'), ('deps', None), ('misc', None)]), OrderedDict([('id', 2), ('form', 'worked'), ('lemma', 'work'), ('upostag', 'VERB'), ('xpostag', 'VERB'), ('feats', None), ('head', None), ('deprel', '_'), ('deps', None), ('misc', None)]), OrderedDict([('id', 3), ('form', 'for'), ('lemma', 'for'), ('upostag', 'ADP'), ('xpostag', 'ADP'), ('feats', None), ('head', None), ('deprel', '_'), ('deps', None), ('misc', None)]), OrderedDict([('id', 4), ('form', 'the'), ('lemma', 'the'), ('upostag', 'DET'), ('xpostag', 'DET'), ('feats', None), ('head', None), ('deprel', '_'), ('deps', None), ('misc', None)]), OrderedDict([('id', 5), ('form', 'BBC'), ('lemma', 'BBC'), ('upostag', 'PROPN'), ('xpostag', 'PROPN'), ('feats', None), ('head', None), ('deprel', '_'), ('deps', None), ('misc', None)]), OrderedDict([('id', 6), ('form', 'for'), ('lemma', 'for'), ('upostag', 'A

[(He --|label|--> ROOT),
 (worked --|label|--> He),
 (for --|label|--> worked),
 (the --|label|--> for),
 (BBC --|label|--> the),
 (BBC --|label|--> for),
 (for --|label|--> a),
 (decade --|label|--> a),
 (decade --|label|--> for),
 (decade --|label|--> BBC),
 (. --|label|--> decade)]

# Constituency Parsing


In [ ]:
class Rule(object):
    
    def __init__(self,
                lhs: str,
                rhs: List[str],
                prob: float=1.0):
        self.lhs = lhs
        self.rhs = rhs
        self.prob = prob
        
    def __repr__(self) -> str:
        return f"'{self.lhs}' -> {self.rhs} [{self.prob}]"

class Grammar(object):
    
    def __init__(self, 
                 start: str,
                 rules: Iterable[Rule],
                 non_terminals: Iterable[str]=None,
                 terminals: Iterable[str]=None):
        # Leave nonterminals and terminals empty to infer them from the rules
        self.start = start
        self.rules = set(rules)
        
        if non_terminals:
            self.non_terminals = set(non_terminals)
        if terminals:
            self.terminals = set(terminals)
            
        if non_terminals is None or terminals is None:
            self.non_terminals = set()
            self.terminals = set()
            
            for rule in self.rules:
                self.non_terminals.add(rule.lhs)
                if len(rule.rhs) == 1:
                    self.terminals.add(rule.rhs[0])
                elif len(rule.rhs) == 2:
                    self.non_terminals.add(rule.rhs[0])
                    self.non_terminals.add(rule.rhs[1])
                else:
                    raise ValueError("Only rules in Chomsky Normal Form supported.")
        
        self.non_terminals.add(start)
        
    def get_lhs_for_rhs(self, rhs: Sequence[str]) -> Set[str]:
        rhs = list(rhs)
        x = set(rule.lhs for rule in
            filter(lambda rule: rule.rhs == rhs, self.rules)
        )
        if x:
            print(list(filter(lambda rule: rule.rhs == rhs, self.rules)))
        return x
                
    def __repr__(self) -> str:
        return f"({self.start}, {self.non_terminals}, {self.terminals}, {self.rules})"

class CYKParser(object):
    
    def __init__(self, grammar: Grammar):
        self.grammar = grammar
        
    def draw_table(self, table: List[List[Set[str]]]) -> str:
        ret = []
        
        for i, row in enumerate(table):
            row_ret = ""
            for j, cell in enumerate(row):
                row_ret += f"[{i},{j}: {cell}]"
            ret.append(row_ret)
        
        return "\n".join(ret)
        
    def recognize(self, sentence: Sequence[str]) -> bool:
        # Implemented like Figure 13.10 in J&M p.474
        
        sentence = list(sentence)
        
        # Fill the table, a list of lists.
        table = [
            [set() for _ in range(len(sentence))]
            for _ in range(len(sentence))
        ]
                
        # Iterate word by word. 
        # Note: The indices may be off by one compared to the 
        # pseudo code in the book due to pythons lists etc.
        for j in range(len(sentence)):
            # Put the POS tag for the current terminal in the diagonal cells
            print()
            table[j][j] = self.grammar.get_lhs_for_rhs([sentence[j]])

            for i in range(j, -1, -1):
                for k in range(i, j):
                    # NT1 describes the span from i to k,
                    # NT2 the one from k to j
                    # If we find something that goes X -> NT1 NT2,
                    # we can conver the span i to j!
                    print(f"j={j} | i={i}")

                    non_terminals_1 = table[i][k]
                    non_terminals_2 = table[k + 1][j]
                    
                    
                    for rhs in itertools.product(non_terminals_1, non_terminals_2):
                        # Find a left hand side from rules that produce
                        # both non-terminals. Add them (set: union) to the table.
                        table[i][j] = table[i][j] | self.grammar.get_lhs_for_rhs(rhs)

            
        print(self.draw_table(table))
        return self.grammar.start in table[0][-1]


In [ ]:
rules = {
    Rule("SS", "S Punct".split()),
    Rule("S", "NP VP".split()),
    Rule("S", "Pron VP".split()),
    Rule("NP", "Det Noun".split()),
    Rule("VP", "Verb VP1".split()),
    Rule("VP1", "PP PP".split()),
    Rule("PP", "Prep NP".split()),
    Rule("PP", "Prep Pron".split()),
    Rule("Noun", ["BBC"]),
    Rule("Noun", ["decade"]),
    Rule("Verb", ["worked"]),
    Rule("Pron", ["He"]),
    Rule("Det", ["the"]),
    Rule("Det", ["a"]),
    Rule("Prep", ["for"]),
    Rule("Punct", ["."])
}

g = Grammar("SS", rules)

p = CYKParser(g)

print(p.recognize("He worked for the BBC for a decade .".split()))